In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import seaborn as sns
import geopandas as gpd
from matplotlib.colors import TwoSlopeNorm  # Use CenteredNorm for newer matplotlib versions
from matplotlib.cm import ScalarMappable
import matplotlib.colors

In [31]:
matplotlib.rc('font', family='Arial') 

In [400]:
# Create a new figure with custom subplot layout
start_year, middle_year1,middle_year2, end_year=2003, 2011, 2011, 2019
fig = plt.figure(figsize=(23, 11), dpi=500)

# Create grid specification for subplots
gs = fig.add_gridspec(21, 50)

ax1 = fig.add_subplot(gs[:12, 0:22])
subax1=fig.add_subplot(gs[1:11, 22])
ax1.text(-0.05, 1.05, 'A', transform=ax1.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')
ax2 = fig.add_subplot(gs[:12, 25:47])
subax2 = fig.add_subplot(gs[1:11, 47])
ax2.text(-0.05, 1.05, 'B', transform=ax2.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')
plotAB(ax1, ax2, subax1, subax2 , start_year, middle_year1,middle_year2, end_year)
ax1.set_xlim(-180, 180)
ax2.set_xlim(-180, 180)



ax3 = fig.add_subplot(gs[12:, 0:10])
ax31 = fig.add_subplot(gs[12:, 10])
ax3.text(-0.05, 1.1, 'C', transform=ax3.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')

ax4 = fig.add_subplot(gs[12:, 13:23])
ax41 = fig.add_subplot(gs[12:, 23])
ax4.text(-0.05, 1.1, 'D', transform=ax4.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')


ax5 = fig.add_subplot(gs[12:, 26:36])
ax51 = fig.add_subplot(gs[12:, 36])

ax5.text(-0.05, 1.1, 'E', transform=ax5.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')



ax6 = fig.add_subplot(gs[12:, 39:49])
ax61 = fig.add_subplot(gs[12:, 49])
ax6.text(-0.05, 1.1, 'F', transform=ax6.transAxes, verticalalignment='top', fontsize=25,fontweight='bold')


Results=CDEF([ax3,ax4,ax5,ax6], start_year, middle_year1,middle_year2, end_year)

plot_subsub([ax31,ax41, ax51, ax61], Results)
# Adjust the layout manually
fig.subplots_adjust(left=0.05, right=0.95, top=1, bottom=0.1, hspace=0.3, wspace=0.3)#plt.tight_layout()
#fig.savefig('Final Figure2-ver3.pdf')

C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\1413322804.py:132: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\1413322804.py:187: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
C:\Users\Yasaman\AppData\Local\Temp\ipykernel_57392\1649085844.py:37: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will b

In [7]:
def plot_subsub(axis, Results):
    for ax, res in zip(axis, Results):
        # Create DataFrame
        df = res.reset_index()
        # Set index to country names for better plotting
        df.set_index('Aff_country', inplace=True)
        norm = matplotlib.colors.LogNorm()  # Normalization for the colormap
        sns.heatmap(df[['sum']], annot=True, fmt=".1f", cmap='viridis', cbar=False, norm=norm, ax=ax)
        ax.set_title('sum')
        ax.set_yticks([])
        ax.set_yticklabels([])
        ax.set_xlabel('')
        ax.set_ylabel('')

In [396]:
def plotAB(ax3, ax4, subax3, subax4, start_year, middle_year1,middle_year2, end_year):  
    # -----------------------------------------
    # 1) Read and preprocess main DataFrame
    # -----------------------------------------
    df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")

    # Rename columns in a single step
    df.rename(
        columns={
            'year': 'Year',
            'aggregated_value': 'count',
            'country': 'Mention_country',
            'affiliation_country': 'Aff_country'
        },
        inplace=True
    )

    # Define dictionary for countries and get lowercase abbreviations
    Country_list = {
        'Egypt': 'EGY', 'Tunisia': 'TUN', 'Libya': 'LBY', 'Syria': 'SYR', 'Yemen': 'YEM',
        'Bahrain': 'BHR', 'Jordan': 'JOR', 'Kuwait': 'KWT', 'Morocco': 'MAR', 'Oman': 'OMN'
    }
    abbr = [code.lower() for code in Country_list.values()]

    # Exclude certain subject areas
    physical_sciences = ['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
    df = df[~df['subjarea'].isin(physical_sciences)]

    # Group, sum, and reset index
    df = df.groupby(['Year', 'Mention_country', 'Aff_country'], as_index=False)['count'].sum()

    # -----------------------------------------
    # 2) Read and prepare country-code mapping
    # -----------------------------------------
    country_codes = pd.read_csv(r"C:\Users\Yasaman\Downloads\iso3.csv")
    # Convert iso3 to lowercase
    country_codes['iso3'] = country_codes['iso3'].str.lower()
    # Build dictionary in one go using set_index + to_dict
    map_dict = country_codes.set_index('iso3')['name'].to_dict()

    # Update manual overrides
    map_dict['irn'] = 'Iran'
    map_dict['usa'] = 'USA'
    map_dict['gbr'] = 'UK'



    def create_period_df(data: pd.DataFrame,
                        year_range: range,
                        abbr_list: list,
                        suffix: str) -> pd.DataFrame:
        """
        Creates a DataFrame for a specific period (e.g. 'before' or 'after'),
        including both total count and 'target' count (mentions of interest).
        """
        # Filter by the given year range
        filtered = data[data['Year'].isin(year_range)]
        
        # Group and sum total counts
        count_df = (
            filtered
            .groupby('Aff_country', as_index=False)['count']
            .sum()
            .rename(columns={'count': f'count_{suffix}'})
        )
        
        # Group and sum 'target' counts (only certain Mention_countries)
        target_df = (
            filtered[
                (filtered['Mention_country'].isin(abbr_list)) &
                (filtered['Mention_country'] != filtered['Aff_country'])
            ]
            .groupby('Aff_country', as_index=False)['count']
            .sum()
            .rename(columns={'count': f'target_count_{suffix}'})
        )
        
        return count_df.merge(target_df, on='Aff_country', how='left')

    # -----------------------------------------
    # 4) Build before_df and after_df
    #    (Assuming start_year, middle_year1, middle_year2, end_year exist)
    # -----------------------------------------
    # Replace np.arange(..., 1) with Python range(...) for cleaner syntax
    before_df = create_period_df(
        data=df,
        year_range=range(start_year, middle_year1),
        abbr_list=abbr,
        suffix='before'
    )

    after_df = create_period_df(
        data=df,
        year_range=range(middle_year2, end_year),
        abbr_list=abbr,
        suffix='after'
    )

    # Merge the two CDF DataFrames
    merged_df = before_df.merge(after_df, on='Aff_country', how='outer')
    merged_df=merged_df[merged_df['Aff_country'].isin(possible_countries)]
    #merged_df=merged_df.fillna(0)
    method='dense'
    na='top'
    merged_df['cdf_after_world']=merged_df['count_after'].rank(method=method, pct=True, na_option=na)
    merged_df['cdf_before_world']=merged_df['count_before'].rank(method=method, pct=True, na_option=na)
    merged_df['cdf_after_target']=merged_df['target_count_after'].rank(method=method, pct=True, na_option=na)
    merged_df['cdf_before_target']=merged_df['target_count_before'].rank(method=method, pct=True, na_option=na)
    merged_df.fillna(0, inplace=True)

    new_df = (
        merged_df
        .assign(
            difference_target=lambda x: x['cdf_after_target'] - x['cdf_before_target'],
            relative_difference=lambda x: (
                (x['difference_target']) / (1 - x['cdf_before_target'])
            ),
            # Normalize target counts by length of each period
            target_count_before=lambda x: x['target_count_before'] / len(range(start_year, middle_year1)),
            target_count_after=lambda x: x['target_count_after'] / len(range(middle_year2, end_year)),
            # difference in average (normalized) target counts
            dif_count=lambda x: x['target_count_after'] - x['target_count_before'],
            # Convert affiliation country code to uppercase
            Aff_country=lambda x: x['Aff_country'].str.upper(),
            # difference in total counts
        )
    )



    # Load the world map
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

    # Assume new_df is a DataFrame with columns 'Aff_country' and 'ratio'
    # Merge your data with the world map
    world = world.merge(new_df, left_on='iso_a3', right_on='Aff_country', how='left')
    # Plot the map
    world.boundary.plot(ax=ax3, color='k', linewidth=0.02)
    norm = matplotlib.colors.LogNorm()  # Normalization for the colormap
    cmap = plt.cm.viridis  # Colormap
    world.plot(column='dif_count', ax=ax3, cmap=cmap, norm=norm,
               missing_kwds={'color': 'lightgrey'})
    
    sm_ax3 = ScalarMappable(norm=norm, cmap=cmap)
    sm_ax3.set_array([])  # Necessary to 'activate' the ScalarMappable for the colorbar
    cbarax3=plt.colorbar(sm_ax3, cax=subax3, orientation='vertical')
    cbarax3.set_label(r"$\Delta A^*_i$", size=18)  # Adjust fontsize here


    # Find the top 5 countries by ratio
    top_countries = world.dropna(subset=['dif_count']).nlargest(5, 'dif_count')

    # Coordinates in the ocean to place annotations and points (manually specified)
    ocean_points = [
        (-160, -5),  # Example coordinates, you may need to adjust these
        (-160, -20),
        (-160, -35),
        (-160, -50),
        (-160, -65)
    ]
    ax3.text(-160, 15, 'Top 5',verticalalignment='top', horizontalalignment='left', fontsize=16)
    top_countries.set_index(top_countries.name, inplace=True)
    top_countries.rename(index={'United States of America':'USA', 'United Kingdom':'UK', 'United Arab Emirates':'UAE'}, inplace=True)
 
    # Plot top countries points with annotations in the ocean
    for idx, (point, row) in enumerate(zip(ocean_points, top_countries.iterrows())):
        _, row = row
        if row['name']=='United States of America':
            name='USA'
        elif row['name']=='United Kingdom':
            name='UK'
        else:
            name=row['name']
        color = cmap(norm(row['dif_count']))  # Get the color from the colormap and normalization
        ax3.scatter(point[0], point[1], color=color, edgecolor='black', s=30)  # Plot point
        ax3.text(point[0]+5, point[1]+2, f"{name}:{row['dif_count']:.1f}",
                 verticalalignment='top', horizontalalignment='left', fontsize=13)

    ax3.set_xticks([])
    ax3.set_yticks([])





    # Load the world map
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    new_df['Aff_country'] = new_df['Aff_country'].apply(lambda x: x.upper())
    # Merge your data with the world map
    world = world.merge(new_df, left_on='iso_a3', right_on='Aff_country', how='left')
    vmin = -1
    vmax = 1
    vcenter = 0
    norm = TwoSlopeNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)
    cmap = plt.cm.PRGn   
    # Plot the map
    world.boundary.plot(ax=ax4, color='k', linewidth=0.02)
    world.plot(column='relative_difference', ax=ax4, cmap=cmap, norm=norm,
               missing_kwds={'color': 'lightgrey'})
    sm_ax3 = ScalarMappable(norm=norm, cmap=cmap)
    sm_ax3.set_array([])  # Necessary to 'activate' the ScalarMappable for the colorbar
    cbar_ax3 =plt.colorbar(sm_ax3, cax=subax4, orientation='vertical' )
    cbar_ax3.set_label(r"$\frac{\Delta q(A^*_i)}{1-q(A^*_i)}$", size=18)
 
    # Find the top 5 countries by ratio
    top_countries = world.dropna(subset=['relative_difference']).nlargest(5, 'relative_difference')

    # Coordinates in the ocean to place annotations and points (manually specified)

    ax4.text(-160, 15, 'Top 5',verticalalignment='top', horizontalalignment='left', fontsize=16)

    # Plot top countries points with annotations in the ocean
    for idx, (point, row) in enumerate(zip(ocean_points, top_countries.iterrows())):
        _, row = row
        if row['name']=='United States of America':
            name='USA'
        elif row['name']=='United Kingdom':
            name='UK'
        else:
            name=row['name']
        color = cmap(norm(row['relative_difference']))  # Get the color from the colormap and normalization
        ax4.scatter(point[0], point[1], color=color, edgecolor='black', s=30)  # Plot point
        ax4.text(point[0] + 5, point[1] + 2, f"{name}:{row['relative_difference']:.2f}",
                 verticalalignment='top', horizontalalignment='left', fontsize=13)

    ax4.set_xticks([])
    ax4.set_yticks([])



In [374]:
top_countries

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Aff_country,count_before,target_count_before,count_after,...,difference_target,relative_difference,dif_count,difference_count,Before_rank_regime_world,After_rank_regime_world,Before_rank_regime_target,After_rank_regime_target,Rank_inertia_world,Rank_inertia_target
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",USA,202574.759242,178.711729,341474.106544,...,0.000000,NaN,167.178095,138899.347302,False,False,False,False,+ +,+ +
143,66834405.0,Europe,United Kingdom,GBR,2829108,"MULTIPOLYGON (((-6.19788 53.86757, -6.95373 54...",GBR,85661.484921,106.395883,132245.319065,...,0.007487,0.546584,112.110141,46583.834143,False,False,False,False,+ +,+ +
158,34268528.0,Asia,Saudi Arabia,SAU,792966,"POLYGON ((34.95604 29.35655, 36.06894 29.19749...",SAU,3219.134545,15.152572,10882.994222,...,0.051774,0.581462,69.921679,7663.859677,False,False,False,False,+ +,+ +
43,67059887.0,Europe,France,FRA,2715518,"MULTIPOLYGON (((-51.65780 4.15623, -52.24934 3...",FRA,30886.617319,125.800639,46595.069623,...,-0.005573,-0.813665,68.583943,15708.452304,False,False,False,False,+ +,+ +
132,47076781.0,Europe,Spain,ESP,1393490,"POLYGON ((-7.45373 37.09779, -7.53711 37.42890...",ESP,28687.109883,43.753352,54667.379068,...,0.009402,0.274534,63.743924,25980.269185,False,False,False,False,+ +,+ +
121,83132799.0,Europe,Germany,DEU,3861123,"POLYGON ((14.11969 53.75703, 14.35332 53.24817...",DEU,39820.212524,78.021239,65381.194718,...,0.001914,0.093168,60.110245,25560.982194,False,False,False,False,+ +,+ +
141,60297396.0,Europe,Italy,ITA,2003576,"MULTIPOLYGON (((10.44270 46.89355, 11.04856 46...",ITA,28679.170995,44.528662,52077.526730,...,-0.003659,-0.133540,49.987597,23398.355735,False,False,False,False,+ +,+ +
148,31949777.0,Asia,Malaysia,MYS,364681,"MULTIPOLYGON (((100.08576 6.46449, 100.25960 6...",MYS,5779.027075,7.152487,25355.400338,...,0.107207,0.711462,44.608824,19576.373263,False,False,False,False,+ +,+ +
107,82913906.0,Asia,Iran,IRN,453996,"POLYGON ((48.56797 29.92678, 48.01457 30.45246...",IRN,12437.860200,7.692239,46616.780172,...,0.061814,0.474992,32.090331,34178.919973,False,False,False,False,+ +,+ +
137,25364307.0,Oceania,Australia,AUS,1396567,"MULTIPOLYGON (((147.68926 -40.80826, 148.28907...",AUS,45616.785259,15.017476,85024.225170,...,0.039990,0.417036,30.460091,39407.439911,False,False,False,False,+ +,+ +


In [22]:
def CDEF(axis, start_year, middle_year1,middle_year2, end_year):
    df=pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
    df=df.rename(columns={'year':'Year', 'aggregated_value':'count', 'country':'Mention_country', 'affiliation_country':'Aff_country'})
    Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
    rev_Country_list={Country_list[key]: key for key in Country_list}
    abbr=[country.lower() for country in Country_list.values()]
    country_codes=pd.read_csv(r"C:\Users\Yasaman\Downloads\iso3.csv")
    country_codes['iso3']=[c.lower() for c in country_codes['iso3']]
    map={country_codes.iloc[c]['iso3']: country_codes.iloc[c]['name'] for c in range(len(country_codes))}
    map['irn']='Iran'
    map['usa']='USA'
    map['gbr']='UK'
    Results=[]
    social_sciences=['PSYC', 'SOCI', 'DECI', 'ARTS', 'ECON', 'BUSI']
    health_sciences=['MEDI', 'NURS', 'DENT', 'HEAL', 'VETE']
    life_sciences=['PHAR', 'NEUR', 'AGRI', 'IMMU', 'BIOC']
    physical_sciences=['CHEM', 'EART', 'ENER', 'CENG',  'ENVI']
    categories={'Social sciences':social_sciences,"Health sciences":health_sciences, 'Life sciences':life_sciences ,"Physical sciences":physical_sciences }
    
    for ax, category_dict_key in zip(axis, categories.keys()):
        category=categories[category_dict_key]
        filtered_df=df[df['subjarea'].isin(category)]
        before_df=filtered_df[filtered_df['Year'].isin(np.arange(start_year, middle_year1, 1))].groupby(by=['Mention_country','Aff_country']).sum().reset_index()[['count','Mention_country','Aff_country']].rename(columns={'count':'count_before'})
        after_df=filtered_df[filtered_df['Year'].isin(np.arange(middle_year2, end_year, 1))].groupby(by=['Mention_country','Aff_country']).sum().reset_index()[['count','Mention_country','Aff_country']].rename(columns={'count':'count_after'})

        compare_df=before_df.merge(after_df, how='outer', on=['Mention_country','Aff_country']).fillna(0)
        compare_df['count_after']/=len(np.arange(start_year, middle_year1, 1))
        compare_df['count_before']/=len(np.arange(middle_year2, end_year, 1))
        compare_df['difference']=compare_df['count_after']-compare_df['count_before']
        compare_df=compare_df[(compare_df['Mention_country'].isin(abbr))&(compare_df['Mention_country']!=compare_df['Aff_country'])]
        compare_df['Mention_country']=compare_df['Mention_country'].map(map)
        compare_df['Aff_country']=compare_df['Aff_country'].map(map)
        heatmap_data = compare_df.drop_duplicates().fillna(0).pivot(index='Aff_country', columns='Mention_country', values='difference')
        heatmap_data=heatmap_data.fillna(0)
        for col in heatmap_data.columns:
            if col in heatmap_data.index:
              heatmap_data.loc[col][col]=np.nan
        row_sums = heatmap_data.sum(axis=1)
        
        heatmap_data['sum']=row_sums
        df_sorted = heatmap_data.loc[row_sums.sort_values(ascending=False).index][:10 :]
        Results.append(df_sorted['sum'])
        df_sorted.rename(index={'United States of America':'USA', 'United Kingdom':'UK', 'United Arab Emirates':'UAE'}, columns={'Syrian Arab Republic':'Syria'}, inplace=True)
        custom_order=list(Country_list.keys())
        df_sorted = df_sorted[custom_order]
        sns.heatmap(df_sorted.iloc[:, : -1], ax=ax, annot=True, cbar=False,center=0, cmap='BrBG',fmt='.1f',annot_kws={"fontsize":10})
        ax.set_title(category_dict_key, fontsize=20)
        ax.set_xlabel('Mentioned country', fontsize=14)
        if len(Results)==1:
            ax.set_ylabel('Affiliation country', fontsize=14)
        else:
            ax.set_ylabel('')
            
    return Results




In [383]:
pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df=pop_df[['Country Code','2003']].dropna()
pop_df['2003']=pop_df['2003'].astype(int)
possible_countries=pop_df.query(" `2003` >=500000")['Country Code'].values
possible_countries=[x.lower() for x in possible_countries]